Report - V1: 18425
Report - V2: 96
The status code is 200 (OK).
Smart meter - Authentication successful...
Logout success.
Report - V3: 365.94
Report - V4: 36


In [23]:
# Report V1. Sum total of all aWattar prices over periods from 00:00 to 24:00=00:00 …. V1 = Sum (Awattar) - DONE
# Report V2. Calculation of average aWattar prices of this period (T) in hours (days * 24) … V2 = V1/T - DONE
# Report V3. Calculation of energy consumed over period T … V3 = Sum (Energy) - DONE
# Report 4. Calculation of aWattar reference costs … Cref = V2 x V3 / 1000 - DONE

# Report 5. Calculation of costs for every period of energy of hour x awattar of hour … c = e * a /1000
# Report 6. Calculation of sum of hourly energy costs … C = Sum (c)
# Report 7. Calculation of average savings in € … S€ = Cref – C
# Report 8. Calculation of average savings in % … S% = (Cref – C) / Cref *100



import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import sqlite3
import math

from external_services.awattar_services import AwattarServices
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.auto_mode import Auto_Mode
from main_services.common_utils import common_utils 

import time
from datetime import datetime
import timedelta

 
def reports():
    # Connect to the SQLite database
    conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
    df = pd.read_sql_query("SELECT * FROM datacache_report WHERE start_timestamp>=date('now', '-2 days') ORDER BY id DESC", conn)
    # Convert DataFrame to JSON
    #json_data = df.to_json(orient='records')
    # print(df)

    # Awattar Prices
    # Assuming you have a DataFrame named 'df' with the provided columns
    # Convert 'start_timestamp' and 'end_timestamp' columns to datetime objects
    df['start_timestamp'] = pd.to_datetime(df['start_timestamp'])
    df['end_timestamp'] = pd.to_datetime(df['end_timestamp'])

    # Calculate the duration of each period in hours
    df['duration_hours'] = (df['end_timestamp'] - df['start_timestamp']).dt.total_seconds() / 3600

    # Convert 'awattar_price' to numeric (in case it's not already)
    df['awattar_price'] = pd.to_numeric(df['awattar_price'], errors='coerce')

    # Calculate the sum of aWattar prices over the specified periods
    total_awattar_price = df['awattar_price'].sum()

    # Calculate the total duration in hours
    total_duration_hours = df['duration_hours'].sum()

    # Calculate the average aWattar price
    average_awattar_price = total_awattar_price / total_duration_hours

    print("V1: Sum total of all aWattar prices over periods: ", total_awattar_price)
    # print("Total duration in hours:", total_duration_hours)
    print("V2: Calculation of average aWattar prices of this period (T): ", average_awattar_price)



    # Smart Meter Data
    # Assuming you have a DataFrame named 'df' with the provided columns
    # Convert 'start_timestamp' and 'end_timestamp' columns to datetime objects
    df['start_timestamp'] = pd.to_datetime(df['start_timestamp'])
    df['end_timestamp'] = pd.to_datetime(df['end_timestamp'])

    # Calculate the duration of each period in hours
    df['duration_hours'] = (df['end_timestamp'] - df['start_timestamp']).dt.total_seconds() / 3600

    # Convert 'awattar_price' to numeric (in case it's not already)
    df['smart_meter_consumption'] = pd.to_numeric(df['smart_meter_consumption'], errors='coerce')

    # Calculate the sum of aWattar prices over the specified periods
    total_smartmeter_consumption = df['smart_meter_consumption'].sum()

    # Calculate the total duration in hours
    total_duration_hours = df['duration_hours'].sum()

    # Calculate the average aWattar price
    average_smartmeter_consumption = total_smartmeter_consumption / total_duration_hours
 
    print("V3: Calculation of energy consumed over period T: ", total_smartmeter_consumption)


    cref = average_awattar_price * total_smartmeter_consumption / 1000
    print("Cref: Calculation of aWattar reference costs Cref: ", cref)
 
    df['result'] = df['awattar_price'] * df['smart_meter_consumption'] / 1000
    
    C = df['result'].sum()
    print("C: Calculation of sum of hourly energy costs: ", C)

    print("S€: Calculation of average savings in €: ", cref - C)

    print("S%: Calculation of average savings in %: ", (cref - C) / cref * 100)


reports()


V1: Sum total of all aWattar prices over periods:  1793.67
V2: Calculation of average aWattar prices of this period (T):  74.73625
V3: Calculation of energy consumed over period T:  37.588
Cref: Calculation of aWattar reference costs Cref:  2.8091861650000003
C: Calculation of sum of hourly energy costs:  2.0728334200000003
S€: Calculation of average savings in €:  0.736352745
S%: Calculation of average savings in %:  26.212315658332315
